In [1]:
import dlib
import cv2
import imutils
import os
import numpy as np
import pandas as pd
import keras
import shutil
import scipy.io
from tqdm import tqdm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array

from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential


target_size = (224, 224)

# fotografta yuz var mi kontrolu
def check_detection(img):
    exact_image = False
    if type(img).__module__ == np.__name__:
        exact_image = True

    base64_img = False
    if len(img) > 11 and img[0:11] == "data:image/":
        base64_img = True


    elif not exact_image:  # image path passed as input

        if not os.path.isfile(img):
            raise ValueError("Confirm that ", img, " exists")

        img = cv2.imread(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects, scores, idx = face_detector.run(gray,1,-1)
            
    if len(rects) > 0:
        return True
    else:
        return False
    
# yuz tespit eden fonksyion
def detect_face(img):
    exact_image = False
    if type(img).__module__ == np.__name__:
        exact_image = True

    base64_img = False
    if len(img) > 11 and img[0:11] == "data:image/":
        base64_img = True

    elif not exact_image:  # image path passed as input
        if not os.path.isfile(img):
            raise ValueError("Confirm that ", img, " exists")

        img = cv2.imread(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # rects: yuz tespiti yapilan cerceve
    # scores: yuz tespiti skorlari
    # best_score_index: en yuksek skora sahip yuzun indisi
    rects, scores, idx = face_detector.run(gray,1,-1)
    best_score_index = np.argmax(scores)
    faces = []
    # tespit edilen yuzler fotograftan kirpiliyor 
    for rect in rects:
        detected_face = img[max(0, rect.top()): min(rect.bottom(), img.shape[0]),
                    max(0, rect.left()): min(rect.right(), img.shape[1])]
        
        # fotograflar 224, 224 hale getiriliyor
        detected_face = cv2.resize(detected_face, (target_size[0], target_size[1]))
        faces.append(detected_face)

    return faces[best_score_index]

# veri setindeki fotograflarin yasini hesaplayan fonksiyon
def datenum_to_datetime(datenum):
    days = datenum % 1

    hours = days % 1 * 24
    minutes = hours % 1 * 60
    seconds = minutes % 1 * 60
    exact_date = datetime.fromordinal(int(datenum)) \
                 + timedelta(days=int(days)) + timedelta(hours=int(hours)) \
                 + timedelta(minutes=int(minutes)) + timedelta(seconds=round(seconds)) \
                 - timedelta(days=366)

    return exact_date.year

# goruntuleri egitim icin uygun formata getiren fonksiyon
def preprocess_pixels(img_pixels):
    img_pixels = np.expand_dims(img_pixels, axis=0)
    return img_pixels

# tahmin olasiliklarini kendi indisleriyle carparak yas tahmini ureten fonksiyon
def findApparentAge(age_predictions):
    output_indexes = np.array([i for i in range(0, 101)])

    apparent_age = np.sum(age_predictions * output_indexes)

    return apparent_age

In [2]:
# verinin okunmasi
mat = scipy.io.loadmat('wiki_crop/wiki.mat')
instances = mat['wiki'][0][0][0].shape[1]

columns = ["dob", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score"]
df = pd.DataFrame(index=range(0, instances), columns=columns)

for i in mat:
    if i == "wiki":
        current_array = mat[i][0][0]
for j in range(len(current_array)):
    df[columns[j]] = pd.DataFrame(current_array[j][0])

df['date_of_birth'] = df['dob'].apply(datenum_to_datetime)
df['age'] = df['photo_taken'] - df['date_of_birth']
# yuzu olmayan fotograflar cikartiliyor
df = df[df['face_score'] != -np.inf]

# birden fazla yuz barindiran fotograflar cikartiliyor
df = df[df['second_face_score'].isna()]

# yuz skoru threshold altinda kalanlar cikartiliyor
df = df[df['face_score'] >= 3]

# cinsiyeti olmayan fotograflar cikartiliyor
df = df[~df['gender'].isna()]

df = df.drop(
    columns=['name', 'face_score', 'second_face_score', 'date_of_birth', 'face_location', 'dob', 'photo_taken'])
# 0 - 100 yas araliginda olmayanlar cikartiliyor
df = df[df['age'] <= 100]
df = df[df['age'] > 0]

paths = []
for index, i in enumerate(df['full_path']):
    paths.append(i[0])
df['path'] = paths
df['path'] = "wiki_crop/" + df['path']
df = df.drop(columns=['full_path'])

In [3]:
df.head()

,gender,age,path
0,1.0,28,wiki_crop/17/10000217_1981-05-05_2009.jpg
2,1.0,60,wiki_crop/12/100012_1948-07-03_2008.jpg
4,0.0,41,wiki_crop/16/10002116_1971-05-31_2012.jpg
5,0.0,52,wiki_crop/02/10002702_1960-11-09_2012.jpg
6,1.0,34,wiki_crop/41/10003541_1937-09-27_1971.jpg


In [5]:
# yuz tespiti icin detektor 
face_detector = dlib.get_frontal_face_detector()

In [5]:
# fotograflarda yuz tespitinin yapilabilir oldugu kontrol ediliyor
tqdm.pandas()
df['detection'] = df['path'].progress_apply(check_detection)

c:\python38\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|████████████████████████████████████████████████████████████████████████████| 22138/22138 [13:08<00:00, 28.08it/s]


In [6]:
# yuz tespiti yapilan ve yapilamayan fotograf sayisi
df['detection'].value_counts()

True     22124
False       14
Name: detection, dtype: int64

In [8]:
# yuz tespiti yapilamayanlar cikartiliyor
df = df[df.detection == True]
df = df.drop(columns='detection')

In [ ]:
classes = 101
target = df['age'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []

for i in range(0, df.shape[0]):
    features.append(df['path'].values[i])

train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.20)

a = len(train_x)
temp_list = []
# veri setinde yuz tespiti yapiliyor
for index, i in enumerate(train_x):
    # gidisat takibi icin adim sayisi yazdiriliyor
    print(f'{index}/{a}')
    
    img = detect_face(i)
    temp_list.append(img)

train_x = temp_list
temp_list2 = []
a = len(test_x)   

for index, i in enumerate(test_x):
    print(f'{index}/{a}')
    
    img = detect_face(i)
    temp_list2.append(img)

test_x = temp_list2

# veriler uygun formata getiriliyor. (bu kisim biraz fazla memory tuketebilir)
train_x = np.array(train_x)/255
test_x = np.array(test_x)/255

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y
                                        , test_size=0.1, random_state=17)

In [28]:
# vgg face modeli
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

model.load_weights('vgg_face_weights.h5')

for layer in model.layers[:-7]:
    layer.trainable = False

# yeniden egitim icin son katmanlar eklendi 
base_model_output = Sequential()
base_model_output = Convolution2D(classes, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)

age_model = Model(inputs=model.input, outputs=base_model_output)

age_model.compile(loss='categorical_crossentropy'
                  , optimizer=keras.optimizers.Adam()
                  , metrics=['accuracy']
                 )

In [30]:
# /models klasoru olmasi gereklidir. Her tur sonunda en iyi model kontrolu yapilarak kaydedilecektir. 
checkpointer = ModelCheckpoint(
    filepath='models/age.hdf5'
    , monitor="val_loss"
    , verbose=1
    , save_best_only=True
    , mode='auto'
)

# patience early stopping icin gerekli,
# val_loss'un azalmasinin kac epoch beklenecegini belirtir. Azalmazsa egitimi durdurur.
patience = 5
epochs = 10

early_stop = EarlyStopping(monitor='val_loss', patience=patience)

score = age_model.fit(
        train_x, train_y
        , epochs=epochs
        , validation_data=(val_x, val_y)
        , callbacks=[checkpointer, early_stop]
    )

Epoch 1/10
498/498 [==============================] - ETA: 0s - loss: 3.7672 - accuracy: 0.0466
Epoch 00001: val_loss improved from inf to 3.62164, saving model to models/age_new.hdf5
498/498 [==============================] - 201s 404ms/step - loss: 3.7672 - accuracy: 0.0466 - val_loss: 3.6216 - val_accuracy: 0.0486
Epoch 2/10
498/498 [==============================] - ETA: 0s - loss: 3.6424 - accuracy: 0.0521
Epoch 00002: val_loss did not improve from 3.62164
498/498 [==============================] - 201s 404ms/step - loss: 3.6424 - accuracy: 0.0521 - val_loss: 3.6234 - val_accuracy: 0.0554
Epoch 3/10
498/498 [==============================] - ETA: 0s - loss: 3.5598 - accuracy: 0.0597
Epoch 00003: val_loss improved from 3.62164 to 3.55173, saving model to models/age_new.hdf5
498/498 [==============================] - 206s 414ms/step - loss: 3.5598 - accuracy: 0.0597 - val_loss: 3.5517 - val_accuracy: 0.0565
Epoch 4/10
498/498 [==============================] - ETA: 0s - loss: 3.4849

In [70]:
# en iyi model tekrar yuleniyor
from keras.models import load_model
age_model = load_model('models/age.hdf5')

In [71]:
score = age_model.evaluate(test_x, test_y, verbose=1)

139/139 [==============================] - 36s 259ms/step - loss: 3.4479 - accuracy: 0.0592


[-5,+5] araligindaki test islemi bu kisimda yapilabilir.

In [ ]:
trues = 0
a = len(test_x)
for index, i in enumerate(test_x):
    print(f'{index}/{a}')

    img_pixels = preprocess_pixels(i)
    age_predictions = age_model.predict(img_pixels)[0, :]
    apparent_age = findApparentAge(age_predictions)
    if apparent_age-5 <= test_y[index].argmax() <= apparent_age+5:
        trues += 1
print(f'acc:{100*trues/len(test_x)}%')
